In [1]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow


sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


## Full Training Set

### Yolo5s Training

In [ ]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 50,
                            'data': 'atrw.yaml',
                            'weights': 'weights/yolov5s.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train-s/train-full/"
                        }
)

### Yolo5m Training

In [2]:
estimator = TensorFlow(entry_point='train.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 640,
                            'batch-size': 16,
                            'epochs': 50,
                            'data': 'atrw.yaml',
                            'weights': 'weights/yolov5m.pt',
                            'output-s3': "s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train-m/train-full/"
                        }
)

### Run Training Job

In [ ]:
estimator.fit({'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/Main',
               'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
               'train': f's3://{bucket}/atrw/detection/train/'})

## Training Subsets

### Fractional Subsets

In [ ]:
%%capture --no-stderr

sets = [2, 4, 8, 16]

for set in sets:
    estimator = TensorFlow(entry_point='train.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 640,
                                'batch-size': 16,
                                'epochs': 50,
                                'data': f'atrw{set}.yaml',
                                'weights': 'weights/yolov5s.pt',
                                'output-s3': f"s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train-s/frac/train{set}"
                            }
    )
    estimator.fit({'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/Main',
                   'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
                   'train': f's3://{bucket}/atrw/detection/train/'})

### Clustering Split

In [ ]:
%%capture --no-stderr

sets = list(range(10))

for set in sets:
    estimator = TensorFlow(entry_point='train.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 640,
                                'batch-size': 16,
                                'epochs': 50,
                                'data': 'atrw.yaml',
                                'weights': 'weights/yolov5s.pt',
                                'output-s3': f"s3://calvinandpogs-ee148/atrw/out/detection/yolov5/train-s/clustering/vgg16places/split{set}"
                            }
    )
    estimator.fit({'annot': f's3://{bucket}/atrw/detection/annotations/yolov5/ImageSets/clustering/split{set}',
                   'label': f's3://{bucket}/atrw/detection/annotations/yolov5/labels',
                   'train': f's3://{bucket}/atrw/detection/train/'})